# 🚀 Getting Started

💡<b> Before running this notebook</b>, ensure you have configured SharePoint, Azure AI Foundry, set up an application for handling API authentication, granted appropriate roles in Microsoft Purview, and set the appropriate configuration parameters. [Steps listed here.](README.md)

## 1. Setup

### 1.1 Install required libraries

In [ ]:
!pip install -r requirements.txt

### 1.2 Load libraries

In [ ]:
import os
# The JSON module could be potentially removed
import json
from azure.identity import ClientSecretCredential
from pyapacheatlas.core import PurviewClient
from purviewautomation import PurviewCollections, ServicePrincipalAuthentication
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from pyapacheatlas.core.typedef import ClassificationTypeDef, EntityTypeDef
# Purview custom libraries
from custom_libs.purview_utils import (
    filesystemFileSampleList,
    listFilesystemFiles,
    getAADToken,
    moveCollection,
    estimateTokens,
    unstructuredDataClassification,
    rollupClassifications,
    loadPurviewAssets,
    applyPurviewClassifications
)
# SharePoint custom libraries
from custom_libs.sharepoint_utils import (
    SharePointUtils,
)

### 1.2 Initialize Environment

Before running this notebook, you must configure certain environment variables. We will now use environment variables to store our configuration. This is a more secure practice as it prevents sensitive data from being accidentally committed and pushed to version control systems.

Create a `.env` file in your project root (use the provided `.env.sample` as a template). [Detailed steps here](README.md)

> 📌 **Note**
> Remember not to commit the .env file to your version control system. Add it to your .gitignore file to prevent it from being tracked.

In [ ]:
# Instantiate the SharePointDataExtractor client
# The client handles the complexities of interacting with SharePoint's REST API, providing an easy-to-use interface for data extraction.
sharepointClient = SharePointUtils()

# Load environment variables from the .env file
sharepointClient.loadEnvFile()

# Retrieve environment variables
azureOpenAIApiKey=os.getenv("AZURE_OPENAI_API_KEY") 
azureOpenAIDeploymentName=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azureOpenAILLMModel=os.getenv("AZURE_OPENAI_LLM_MODEL")
azureOpenAIApiEndpoint= os.getenv("AZURE_OPENAI_ENDPOINT")
azureOpenAIApiVersion= os.getenv("AZURE_OPENAI_API_VERSION")
purviewAccountName = os.getenv("PURVIEW_ACCOUNT_NAME")
purviewEndpointUrl=os.getenv("PURVIEW_ENDPOINT_URL")
purviewTokenUrl=os.getenv("PURVIEW_TOKEN_URL")
tenantId=os.getenv("AZURE_TENANT_ID")
clientId=os.getenv("AZURE_CLIENT_ID")
clientSecret=os.getenv("AZURE_CLIENT_SECRET")
siteDomain = os.getenv("SITE_DOMAIN")
siteName = os.getenv("SITE_NAME")

You will need to update the values for the cell below to match the characteristics of your environment.

In [ ]:
# Enable or disable display of variables
displayVariables = True

# Global variable definitions
fileExtensions = ["docx","pdf","pptx"]
sharepointPath="/Insurance/Claims"
filesystemPath = r"SampleFiles"

# Number of characters to be analyzed by Large Language Model (LLM) from each file
textLength=800

# Sample size for filesystem and SharePoint files
sampleSize=0

# Entity types for classification
entityTypes = ['SharePoint','FileSystem']

# List of custom classifications to be created in Purview
# This list can be customized based on the specific needs of the organization or project.
classifications=[
    "Empty Content", 
    "Insurance Claim",  
    "Sales Receipt",  
    "Insurance Policy",
    "Report",
    "Invoice",
    "PII",
    "Other"
]
# Convert classification list to string
classificationsStr = ''.join(classification+'\n' for classification in classifications)

In [ ]:
if displayVariables:
    print(f"Tenant ID: {tenantId}")
    print(f"Client ID: {clientId}") 
    print(f"Azure OpenAI API Key: {azureOpenAIApiKey}")
    print(f"Azure OpenAI Endpoint: {azureOpenAIApiEndpoint}")

In [ ]:
if not tenantId or not clientId or not clientSecret or not azureOpenAIApiKey:
    raise ValueError("Azure credentials are not set in the environment variables.")

# Generate token for REST API calls
token = getAADToken(tenantId,clientId, clientSecret,purviewTokenUrl)

# Authenticate with Microsoft Graph API
response = sharepointClient.msgraph_auth()

# Generate authentication credentials for Service Principal and Atlas client authentication for different Purview functions
servicePrincipalAuth = ServicePrincipalAuthentication(
    tenant_id=tenantId,
    client_id=clientId,
    client_secret=clientSecret
)

clientCredential = ClientSecretCredential(
    tenant_id=tenantId,
    client_id=clientId,
    client_secret=clientSecret
)

# Create clients for Purview administration and Azure AI Foundry
purviewClient = PurviewClient(
    account_name = purviewAccountName,
    authentication = clientCredential
)

collectionClient = PurviewCollections(
    purview_account_name=purviewAccountName,
    auth = servicePrincipalAuth
)

llmClient = ChatCompletionsClient(
    endpoint=azureOpenAIApiEndpoint,
    credential=AzureKeyCredential(azureOpenAIApiKey),
    temperature=0
)

### 1.4 Create Purview asset dependencies

Creates entity type definitions and classifications required by the Purview clients to assign classifications to assets discovered.

In [ ]:
# Creation of custom Entity Types, required by the custom Classifications
# The list of Entity Types is taken from the variable named entityTypes
for entityName in entityTypes:
    edef = EntityTypeDef(
        name = entityName,
        superTypes= ['DataSet']
    )
    results = purviewClient.upload_typedefs(
        entityDefs=[edef],
        force_update=True
    )

# Creation of custom Classifications
# The list of classifications is taken from the variable named classifications
for classification in classifications:
    # Create custom classifications to be applied to unstructured data assets
    cdef = ClassificationTypeDef(
        name=classification,
        # You need to define the assets type that will be associated with each classification ahead of time.
        # entityTypes will restrict the types of assets that can be associated with this classification.
        # For example: If the asset has a type of FileSystem and the Classification has entityTypes=['DataSet'],
        #              the attempt to classify the asset will fail.
        # entityTypes=['SharePoint','FileSystem','DataSet','Process']
        entityTypes=entityTypes
    )
    # Do the upload
    results = purviewClient.upload_typedefs(
        classificationDefs=[cdef],
        force_update=True
    )

### 1.5 Create custom collections

Creates multiple custom collection under the parent Start_Collection (Domain)


In [ ]:
# To create multiple collections, the parent collection defined by the start_collection parameter
# MUST exist.
response = collectionClient.create_collections(start_collection=purviewAccountName,
                          collection_names=['Unstructured/SharePoint','Unstructured/FileSystem'])

### 1.6 Capture Sampling Size

This will help to determine the number of files that will be analyzed for classification purposes.

> 📌 **Note:**
> Currently is a fixed size, but it could be changed to represent a percentage of the total number of files found during the scan.

In [ ]:
sampleSize = input(f"Enter how many documents to analyze: ")
if sampleSize.isnumeric():
    sampleSize = int(sampleSize)
else:
    sampleSize = 0
print(f"\n{sampleSize} documents will be analyzed from the list of documents found.")

## 2. SharePoint Demo

### 2.1 Scan SharePoint Site

In [ ]:
"""
List all the files in SharePoint site that match the defined file extensions. 
"""
spFileList = sharepointClient.listSharepointFiles(
    site_domain=siteDomain,
    site_name=siteName,
    file_formats = fileExtensions,
    folder_path=sharepointPath,
    # Files modified N minutes ago
    # minutes_ago=60,
)
print(f"{len(spFileList)} files found matching the patterns {fileExtensions}: \n")

In [ ]:
if displayVariables == True:
    print(json.dumps(spFileList, indent=2))

### 2.2 Generate file subset

In [ ]:
# Create a subset of the spFileList based on the number specified by sampleSize. If no subset is provided, the entire list will be used.
if sampleSize == 0 or sampleSize > len(spFileList):
        sampleSize = len(spFileList)
# Create a subset of the SharePoint file list
spFileSubset = sharepointClient.sharepointFileSampleList(spFileList,sampleSize)

In [ ]:
if displayVariables:
    print(f"\nSubset of SharePoint files to be analyzed: {sampleSize} files\n")
    for file in spFileSubset:
        print(f"{file}")

### 2.3 Extract file contents

In [ ]:
"""
Extract file contents and process all file information included in the subset from a 
specific Site ID.
"""
spFileContent = sharepointClient.getSharepointFileContent(
    site_domain=os.environ["SITE_DOMAIN"],
    site_name=os.environ["SITE_NAME"],
    folder_path=sharepointPath,
    file_names=spFileSubset
    # Files modified N minutes ago
    # minutes_ago=60,
)

In [ ]:
if displayVariables:
    print(json.dumps(spFileContent, indent=2))

### 2.4 Analyze File Contents with LLM

### Estimate the number of tokens that will be used by LLM model, prior to processing the documents

In [ ]:
tokens = estimateTokens(spFileContent,textLength,classificationsStr,azureOpenAILLMModel)
print(f"Estimated Number of Tokens: {tokens}")

### 2.5 Classify document contents using LLM

In [ ]:
"""
Analyze SharePoint folder contents using Large Language Model to determine applicable
classifications. 
"""
spFileContent = unstructuredDataClassification(spFileContent,textLength,llmClient,azureOpenAIDeploymentName,classificationsStr)

### 2.6 Organize and Rollup Classifications

In [ ]:
"""
Collect document classifications identified for SharePoint folder
"""
spClassifications = rollupClassifications(spFileContent)


In [ ]:
if displayVariables:
    print(f"\nClassifications for SharePoint files: {spClassifications}")

### 2.7 Ingest assets into Purview via Atlas API

In [ ]:
"""
Load SharePoint Assets in Purview.
"""
spGuids = loadPurviewAssets(purviewClient,spFileContent)

In [ ]:
spGuids[0]

### 2.8 Apply classifications to assets

In [ ]:
"""
Apply classification to SharePoint assets
"""
result = applyPurviewClassifications(purviewClient,spGuids,spClassifications)

### 2.9 Move assets to their final collection

In [ ]:
"""
Move assets from default (root) collection to collectionName
"""
collectionName = 'SharePoint'
output = moveCollection(collectionName,purviewEndpointUrl,token,spGuids)

## 3. File System Demo

### 3.1 Scan Filesystem

In [ ]:
"""
List all the files in Filesystem that match the defined file extensions. 
"""
fsFileList = listFilesystemFiles(filesystemPath, fileExtensions)

In [ ]:
if displayVariables:
    for file in fsFileList:
        print(f"{file}")

### 3.2 Generate file subset and extract contents

In [ ]:
"""
Create a subset of the fsFileList based on the number specified by sampleSize, extract file 
contents, and metadata.
"""
if sampleSize == 0 or sampleSize > len(spFileList):
        sampleSize = len(spFileList)

fsFileContent = filesystemFileSampleList(fsFileList,sampleSize,filesystemPath)

In [ ]:
fsFileContent

### 3.3 Estimate number of tokens to be used by LLM

In [ ]:
tokens = estimateTokens(fsFileContent,textLength,classificationsStr,azureOpenAILLMModel)
print(f"Estimated Number of Tokens: {tokens}")

### 3.4 Classify document contents using LLM

In [ ]:
"""
Analyze Filesystem folder contents using Large Language Model to determine applicable
classifications. 
"""
fsFileContent = unstructuredDataClassification(fsFileContent,textLength,llmClient,azureOpenAIDeploymentName,classificationsStr)

### 3.5 Organize and Rollup Classifications

In [ ]:
"""
Collect document classifications identified for FileSystem folder
"""
fsClassifications = rollupClassifications(fsFileContent)


In [ ]:
if displayVariables:
    print(f"\nClassifications for FileSystem files: {fsClassifications}")

### 3.6 Ingest assets into Purview via Atlas API

In [ ]:
"""
Load FileSystem Assets in Purview.
"""
fsGuids = loadPurviewAssets(purviewClient,fsFileContent)

In [ ]:
if displayVariables:
    print(f"\nFileSystem GUIDs: {fsGuids}")

### 3.7 Apply classifications to assets

In [ ]:
"""
Apply classification to SharePoint assets
"""
result = applyPurviewClassifications(purviewClient,fsGuids,fsClassifications)

### 3.8 Move assets to their final collection

In [ ]:
"""
Move collections from default (root) collection to collectionName
"""
collectionName = 'FileSystem'
output = moveCollection(collectionName,purviewEndpointUrl,token,fsGuids)

## 4. Cleanup section


### 4.1 Delete assets and collections

You can delete individual assets using their respective GUIDs or you can leverage the collectionClient to delete collections recursively.

In [ ]:
# Delete Entities
for guid in [*fsGuids, *spGuids]:
    response = purviewClient.delete_entity(guid=guid)
    print(json.dumps(response, indent=2))

In [ ]:
# Delete sub-collection contents and sub-collections
collectionClient.delete_collections_recursively("Unstructured",delete_assets=True)
# Delete parent collection
collectionClient.delete_collections("Unstructured")

### 4.2 Delete custom classifications and entity types

In [ ]:
# Delete custom classifications
for classification in classifications:
    purviewClient.delete_type(classification)

# Delete custom Entity Types
for entityName in entityTypes:
    # if entityName == 'FileSystem':
    edef = EntityTypeDef(
        name = entityName,
        superTypes= ['DataSet']
    )
    results = purviewClient.delete_typedefs(
        entityDefs=[edef],
        force_update=True
    )

In [ ]:
# Delete all Jupyter notebook variables
%reset -f